### We create a classification dataset and we use the Sklearn linear SVM to see if we get the same weights

In [21]:
import numpy as np
from sklearn.datasets.samples_generator import make_blobs
# generate 2d classification dataset
X, y = make_blobs(n_samples=1000, centers=2, n_features=20)
np.place(y, y==0, -1)

In [26]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X, y)
sklearn_coefs = clf.coef_

### Paper 1

Consider:
- $x_1,..., x_n$ $\in$ $\mathbb{R}^d$
- $y_1,...,y_n$ $\in$ $(-1, 1)$
- $\lambda > 0$ a regularization parameter
- $\phi_1,...,\phi_n$ a sequence of scalars convex functions
- Since we are working on the SVM problem (with linear kernels and no bias term), we set $\phi_i(a) = max(0,1-y_ia)$ (Hinge loss)

For solving SVM, there are 2 approaches:
- stochastic gradient descent (SGD), which aims to minimize the following primal problem:
\begin{equation}
min_{w \text{ } \in \text{ } R^d} \big[ \frac{1}{n} \sum\limits_{i=1}^n \phi_i(w^Tx_i) + \frac{\lambda}{2}||w||^2\big]
\end{equation}
- dual coordinate ascent (DCA), which aims to maximize the following dual problem:
\begin{equation}
max_{\alpha \text{ } \in \text{ } R^n} \big[ \frac{1}{n} \sum\limits_{i=1}^n -\phi_i^*(-\alpha_i) - \frac{\lambda}{2} \big|\big|\frac{1}{\lambda n}\sum\limits_{i=1}^n \alpha_i x_i \big|\big|^2\big]
\end{equation}
here, for each $i$, $\phi_i^*: \mathbb{R} \rightarrow \mathbb{R}$ is the convex conjugate of $\phi_i$, namely $\phi_i^*(u) = max_z(zu - \phi_i(z))$

If we define, $w(\alpha) = \frac{1}{\lambda n}\sum\limits_{i=1}^n \alpha_i x_i$, then we have that $w(\alpha^*) = w^*$ where $\alpha^*$ is an optimal solution of the dual problem.

***
<center>___Procedure SDCA($\alpha^{(0)}$)___</center>
***
___Let___ $w^{(0)} = w(\alpha^{(0)})$<br>
___Iterate:___ for $t=1,2,...,T:$
> Randomly pick $i$<br>
Find $\Delta\alpha_i$ to maximize $-\phi_i^*(-(\alpha_i^{(t-1)}+\Delta\alpha_i)) - \frac{\lambda n}{2} ||w^{(t-1)}+(\lambda n)^{-1} \Delta\alpha_ix_i||^2$<br>
$\alpha^{(t)} \leftarrow \alpha^{(t-1)}+\Delta\alpha_ie_i$<br>
$w^{(t)} \leftarrow w^{(t-1)}+(\lambda n)^{-1}\Delta\alpha_ix_i$<br>
 
___Output (Averaging option):___<br>
> Let $\bar{\alpha} = \frac{1}{T-T_0}\sum\limits_{i=T_0+1}^T \alpha^{(t-1)}$<br>
Let $\bar{w} = w(\bar{\alpha}) = \frac{1}{T-T_0}\sum\limits_{i=T_0+1}^T w^{(t-1)}$<br>
return $\bar{w}$<br>

___Output (Random option):___<br>
> Let $\bar{\alpha} = \alpha^{(t)}$ and $\bar{w} = w^{(t)}$ for some random $t \in T_0+1,...T$<br>
return $\bar{w}$

***

___Close formula to maximize $-\phi_i^*(-(\alpha_i^{(t-1)}+\Delta\alpha_i)) - \frac{\lambda n}{2} ||w^{(t-1)}+(\lambda n)^{-1} \Delta\alpha_ix_i||^2$:___

We want to $max_{\alpha \text{ } \in \text{ } R^n} D(\alpha)$ with $D(\alpha) = \frac{1}{n} \sum\limits_{i=1}^n -\phi_i^*(-\alpha_i) - \frac{\lambda}{2} \big|\big|\frac{1}{\lambda n}\sum\limits_{i=1}^n \alpha_i x_i \big|\big|^2 = - \frac{1}{2\lambda n^2} \alpha^T X^T X \alpha + \frac{1}{n} \sum\limits_{i=1}^n -\phi_i^*(-\alpha_i)$

At each iteration, we have to maximize the updated dual objective defined as:
\begin{equation}
\begin{aligned}
D(\alpha_t+\Delta\alpha_i e_i) = &\frac{-1}{2\lambda n^2} (\alpha_t+\Delta\alpha_i e_i)^T X^T X (\alpha_t+\Delta\alpha_i e_i) - \frac{1}{n}\sum\limits_{i=1}^n \phi_i^*(-\alpha_t-\Delta\alpha_i e_i)\\
=& \frac{-1}{2\lambda n^2} \alpha_t^T X^T X \alpha_t - \frac{1}{\lambda n^2} \alpha_t^T X^T X \Delta\alpha_ie_i - \frac{1}{2\lambda n^2} (\Delta\alpha_i e_i)^T X^T X (\Delta\alpha_i e_i) - \frac{1}{n}\sum\limits_{i=1}^n \phi_i^*(-\alpha_t-\Delta\alpha_i e_i)\\
\end{aligned}
\end{equation}

By setting:
\begin{equation}
\begin{aligned}
Constant =& \frac{-1}{2\lambda n^2} \alpha_t^T X^T X \alpha_t - \frac{1}{n}\sum\limits_{\substack{i=1 \\ i\neq j}}^n \phi_j^*(-\alpha_t-\Delta\alpha_j e_j)\\
A =& \frac{1}{\lambda n} x_i^T x_i = \frac{1}{\lambda n}||x_i||^2\\
B=& x_i^T \frac{X \alpha_t}{\lambda n} = x_i^T w_t
\end{aligned}
\end{equation}

we get:
\begin{equation}
\begin{aligned}
D(\alpha_t+\Delta\alpha_i e_i) \propto \frac{-A}{2} (\Delta\alpha_i)^2 - B \Delta\alpha_i - \phi^*_i(-\alpha_i\Delta\alpha_i)
\end{aligned}
\end{equation}

We recall that in the case of SVM, we use the Hinge loss, i.e.:
\begin{equation}
\phi_i(a) = max(0,1-y_ia)
\end{equation}
and its conjugate is given by:
\begin{equation}
\phi^*_i(u) =
    \begin{cases}
      y_i u & \text{if} -1\leq y_iu\leq 0\\
      +\infty & \text{otherwise}
    \end{cases} 
\end{equation}

Then, considering that $-1\leq y_iu\leq 0$, we can maximize $D(\alpha_t+\Delta\alpha_i e_i)$ by $\Delta\alpha_i$ by:
\begin{equation}
\tilde{\Delta\alpha_i} = \frac{y_i -B}{A}
\end{equation}

Finally, if we incorporate the constraint $-1\leq -y_i(\alpha_i + \Delta\alpha_i)\leq 0 \leftrightarrow 0\leq y_i(\alpha_i + \Delta\alpha_i)\leq 1$, we obtain the update:
\begin{equation}
\Delta\alpha_i = \frac{1}{y_i}max(0, min(1, y_i(\alpha_i + \tilde{\Delta\alpha_i}))) - \alpha_i
\end{equation}

In [52]:
def dual_primal(alpha, X, lambda_):
    return X.T.dot(alpha)/(lambda_*X.shape[0])

In [53]:
from tqdm import tqdm

def SDCA(nb_iteration, X, y, lambda_ = 1, output = 'averaging'):
    n = X.shape[0]
    alpha = [np.zeros(n)]
    w = [dual_primal(alpha[0], X, lambda_)]
    for t in tqdm(range(nb_iteration)):
        i = np.random.randint(0, n)
        A = (1/(lambda_*n))*np.linalg.norm(X[i], 2)**2
        B = X[i].dot(w[t])
        delta_alpha = (y[i]-B)/A
        delta_alpha_constrained = (1/y[i])*max(0, min(1, y[i]*(delta_alpha+alpha[t][i]))) - alpha[t][i]
        e_i = np.zeros(n)
        e_i[i] = 1.
        alpha.append(alpha[t]+delta_alpha_constrained*e_i)
        w.append(dual_primal(alpha[t+1], X, lambda_))
    if output == 'averaging':
        return np.mean(w[int(nb_iteration/2):], axis = 0)
    else:
        return w[np.random.randint(int(nb_iteration/2), nb_iteration)]

In [56]:
sdca_coefs = SDCA(100000, X, y, 1, output = 'averaging')

100%|██████████| 100000/100000 [00:07<00:00, 13091.53it/s]


In [57]:
np.linalg.norm(sdca_coefs - sklearn_coefs, 2)

0.0081328491042149797

***
<center>___Procedure Modified-SGD___</center>
***
___Initialize:___ $w^{(0)} = 0$<br>
___Iterate:___ for $t=1,2,...,n:$<br>
> Find $\alpha_t$ to maximize $-\phi_t^*(-\alpha_t) - \frac{\lambda t}{2} ||w^{(t-1)}+(\lambda t)^{-1} \alpha_tx_t||^2$<br>
Let $w^{(t)} = \frac{1}{\lambda t}\sum\limits_{i=1}^t \alpha_ix_i$<br>
return $\alpha$

***

***
<center>___Procedure SDCA with SGD Initialization___</center>
***
___Stage 1:___ call Procedure Modified-SGD and obtain $\alpha$<br>
___Stage 2:___ call Procedure SDCA with parameter $\alpha^{(0)} = \alpha$<br>
***

***
<center>___Procedure SDCA-Perm($\alpha^{(0)}$)___</center>
***
___Let___ $w^{(0)} = w(\alpha^{(0)})$<br>
___Let___ $t=0$<br>
___Iterate:___ for epoch $k=1,2,...$<br>
> ___Let___ ${i_1,...,i_n}$ be a random permutation of ${1,...,n}$<br>
___Iterate:___ for $j=1,2,...,n:$<br>
&nbsp;&nbsp;&nbsp;&nbsp; $t \leftarrow t+1$<br>
&nbsp;&nbsp;&nbsp;&nbsp; $i=i_j$<br>
&nbsp;&nbsp;&nbsp;&nbsp; Find $\Delta\alpha_i$ to increase dual<br>
&nbsp;&nbsp;&nbsp;&nbsp; $\alpha^{(t)} \leftarrow \alpha^{(t-1)}+\Delta\alpha_ie_i$<br>
&nbsp;&nbsp;&nbsp;&nbsp; $w^{(t)} \leftarrow w^{(t-1)}+(\lambda n)^{-1}\Delta\alpha_ix_i$<br>
 
___Output (Averaging option):___<br>
> Let $\bar{\alpha} = \frac{1}{T-T_0}\sum\limits_{i=T_0+1}^T \alpha^{(t-1)}$<br>
Let $\bar{w} = w(\bar{\alpha}) = \frac{1}{T-T_0}\sum\limits_{i=T_0+1}^T w^{(t-1)}$<br>
return $\bar{w}$<br>

___Output (Random option):___<br>
> Let $\bar{\alpha} = \alpha^{(t)}$ and $\bar{w} = w^{(t)}$ for some random $t \in T_0+1,...T$<br>
return $\bar{w}$

***

In [49]:
from sklearn.utils import shuffle

def SDCA_perm(nb_epoch, X, y, lambda_ = 1, output = 'averaging'):
    n = X.shape[0]
    alpha = [np.zeros(n)]
    w = [dual_primal(alpha[0], X, lambda_)]
    for k in tqdm(range(nb_epoch)):
        X_epo, y_epo = shuffle(X, y)
        for i in range(n):
            A = (1/(lambda_*n))*np.linalg.norm(X_epo[i], 2)**2
            B = X_epo[i].dot(w[k*n+i])
            delta_alpha = (y_epo[i]-B)/A
            delta_alpha_constrained = (1/y_epo[i])*max(0, min(1, y_epo[i]*(delta_alpha+alpha[k*n+i][i]))) - alpha[k*n+i][i]
            e_i = np.zeros(n)
            e_i[i] = 1.
            alpha.append(alpha[k*n+i]+delta_alpha_constrained*e_i)
            w.append(dual_primal(alpha[k*n+i+1], X_epo, lambda_))
    if output == 'averaging':
        return np.mean(w[int(nb_epoch*n/2):], axis = 0)
    else:
        return w[np.random.randint(int(nb_epoch*n/2), nb_epoch*n)]

In [50]:
SDCA_perm_coefs = SDCA_perm(100, X, y, lambda_ = 1, output = 'averaging')

100%|██████████| 100/100 [00:06<00:00, 15.21it/s]


In [51]:
np.linalg.norm(SDCA_perm_coefs - sklearn_coefs, 2)

0.012413574629476157

### Paper 2

The Pegasos algorithm performs stochastic gradient descent on the primal objective.

***
<center>___The basic Pegasos algorithm___</center>
***
___Let___ $w^{(0)} = 0$<br>
___Iterate:___ for $t=1,2,...,T:$
> Randomly pick $i$<br>
Set $\eta_t = \frac{1}{\lambda t}$<br>
If $y_i \langle\, w_{t-1},x_i\rangle < 1$:<br>
&nbsp;&nbsp;&nbsp;&nbsp; $w_t \leftarrow (1-\eta_t \lambda)w_{t-1} + \eta_t y_i x_i$<br>
Else:<br>
&nbsp;&nbsp;&nbsp;&nbsp; $w_t \leftarrow (1-\eta_t \lambda)w_{t-1}$<br>
[Optional: $w_t \leftarrow min\big\{1, \frac{1/\sqrt{\lambda}}{||w_t||}\big\}w_t$]

 
___Output:___<br>
> return $w_T$

***

To use this algorithm, we need to compute the subgradient of the penalized hinge loss:
\begin{equation}
f(w; i_t) = \frac{\lambda}{2}||w||^2 + max(0, 1-y_{i_t} \langle\,w, x_{i_t}\rangle)
\end{equation}

It is givent by:
\begin{equation}
\bigtriangledown_t =
    \begin{cases}
      \lambda w_t - y_{i_t} x_{i_t} & \text{if } y_{i_t} \langle\,w, x_{i_t}\rangle < 1 \\
      \lambda w_t  & \text{otherwise}
    \end{cases} 
\end{equation}

We can add the optional projection step to the previous algorithm to limit the set of admissible solutions to the ball of radius $\frac{1}{\sqrt{\lambda}}$. However, in the experiments, there is no major differences between the projected and unprojected variants of Pegasos.

In [75]:
def Pegasos(nb_iteration, X, y, lambda_ = 1, projection = False):
    n = X.shape[0]
    w = [np.repeat(0, X.shape[1])]
    for t in tqdm(range(1,nb_iteration+1)):
        i = np.random.randint(0, n)
        eta = 1/(lambda_ * t)
        if y[i]*w[t-1].dot(X[i]) < 1:
            w_inter = (1-eta*lambda_)*w[t-1] + eta * y[i] * X[i]
        else:
            w_inter = (1-eta*lambda_)*w[t-1]
        if projection:
            w.append(min(1, (1/np.sqrt(lambda_))/np.linalg.norm(w_inter, 2))*w_inter)
        else:
            w.append(w_inter)
    return w[-1]

In [78]:
Pegasos_coefs = Pegasos(100000, X, y, lambda_ = 1, projection = False)

100%|██████████| 100000/100000 [00:01<00:00, 67725.76it/s]


In [79]:
np.linalg.norm(Pegasos_coefs - sklearn_coefs, 2)

0.0084390124220156144

A mini-batch setting of this algorithm can be used by using k examples at each iteration, where 1 ≤ k ≤ n is a parameter that needs to be provided to the algorithm.

***
<center>___The mini-batch Pegasos algorithm___</center>
***
___Let___ $w^{(0)} = 0$<br>
___Iterate:___ for $t=1,2,...,T:$
> Randomly pick $A_t \subseteq [n]$, where $|A_t|=k$<br>
Set $A_t^+ = \{i \in A_t:y_i \langle\, w_{t-1},x_i\rangle < 1 \}$ <br>
Set $\eta_t = \frac{1}{\lambda t}$<br>
Set $w_t \leftarrow (1-\eta_t \lambda)w_{t-1} + \frac{\eta_t}{k}\sum\limits_{i \in A_t^+} y_i x_i$<br>
[Optional: $w_t \leftarrow min\big\{1, \frac{1/\sqrt{\lambda}}{||w_t||}\big\}w_t$]

 
___Output:___<br>
> return $w_T$

***

In the above description we refer to $A_t$ as chosen uniformly at random among the subsets of $[n]$ of size $k$, i.e. chosen without repetitions. Notice that the analysis still holds when $A_t$ is a multi-set chosen i.i.d. with repetitions.

***
<center>___The kernalized Pegasos algorithm___</center>
***
___Let___ $\alpha^{(0)} = 0$<br>
___Iterate:___ for $t=1,2,...,T:$
>  Randomly pick $i$<br>
For all $j\neq i$:<br>
&nbsp;&nbsp;&nbsp;&nbsp; Set $\alpha_t[j] = \alpha_{t-1}[j]$<br>
If $y_i \frac{1}{\lambda t}\sum\limits_j \alpha_{t-1}[j]y_i K(x_i,x_j) < 1:$<br>
&nbsp;&nbsp;&nbsp;&nbsp; Set $\alpha_t[i] = \alpha_{t-1}[i]+1$<br>
Else:<br>
&nbsp;&nbsp;&nbsp;&nbsp; Set $\alpha_t[i] = \alpha_{t-1}[i]$<br>
 
___Output:___<br>
> return $\alpha_T$

***